In [ ]:
import requests
from elasticsearch import Elasticsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(
    index=index_name,
    body=index_settings
)

In [ ]:
for document in documents:
    es_client.index(
        index=index_name,
        document=document
    )

In [ ]:
query = 'How do copy a file to a Docker container?'

In [ ]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
results = elastic_search(query)

In [ ]:
results

In [ ]:
def build_prompt(query, search_results):

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = "".strip()

    for doc in search_results:
        context_template = context_template + f"Q: {doc['question']}\nA: {doc['text']}\n\n"

    prompt = prompt_template.format(
        question=query,
        context=context_template
    ).strip()

    return prompt

In [ ]:
prompt = build_prompt(query, results)

In [ ]:
print(prompt)
print(len(prompt))

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
print(encoding.encode(prompt))
print(f"\ntokens: {len(encoding.encode(prompt))}")

In [ ]:
encoding.decode_single_token_bytes(63842)

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "OPENAI_API_KEY= environment variable is not set.")

In [ ]:
client = OpenAI()

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )

    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [ ]:
print(rag(query))